In [ ]:
import io
import matplotlib.pyplot as plt
import pandas as pd
import requests

In [ ]:
r = requests.post(
    "https://api.fink-portal.org/api/v1/objects",
    json={"objectId": "ZTF25abcwzci", "output-format": "json"},
)
pdf = pd.read_json(io.BytesIO(r.content))
# rename TOM MOP OPM
namedic = {1: "ZTF_g", 2: "ZTF_r"}
pdf["filter"] = pdf["i:fid"].map(namedic)
new_pdf = pdf[["i:jd", "filter", "i:magpsf", "i:sigmapsf", "d:mulens"]].copy()
new_pdf = new_pdf.rename(
    columns={"i:jd": "time", "i:magpsf": "magnitude", "i:sigmapsf": "error"}
)
# new_pdf.to_csv("{}.csv".format(pdf["i:objectId"][0]), index=False)

In [ ]:
plt.gca().invert_yaxis()
for fid in range(1, 3):
    mask_filt = new_pdf["filter"] == namedic[fid]
    plt.errorbar(
        new_pdf["time"][mask_filt].array,
        new_pdf["magnitude"][mask_filt].array,
        yerr=new_pdf["error"][mask_filt].array,
        fmt=".",
        label=namedic[fid],
    )
    plt.scatter(
        new_pdf["time"][mask_filt].array,
        new_pdf["magnitude"][mask_filt].array,
        c=new_pdf["d:mulens"][mask_filt].array,
        s=70,
        cmap="jet",
        zorder=0,
        alpha=0.5,
    )
cb = plt.colorbar()
plt.legend()

In [ ]:
# %config InlineBackend.figure_formats = ['svg']
import numpy as np
from iminuit import Minuit, cost
from matplotlib import pyplot as plt
from numba_stats import bernstein, norm, t, truncexpon

In [ ]:
def pspl_no_blend(x, log10u0, log10te, t0, fs,fb ):
    u0 = 10**log10u0
    te = 10**log10te
    usqr = u0**2 + (x - t0) ** 2 / te**2
    return fs * (usqr + 2) / (usqr * (usqr + 4.0)) ** 0.5 + fb

flux = 10.0 ** (-0.4 * new_pdf["magnitude"][mask_filt].array)
err_flux = np.abs(new_pdf["error"][mask_filt].array * flux * np.log(10) / 2.5)
guess = (
    np.log10(1),
    np.log10(30.0),
    new_pdf["time"][mask_filt].array[np.argmax(new_pdf["time"][mask_filt].array)],
    np.median(flux),np.min(flux)
)
print(guess)
c = cost.LeastSquares(new_pdf["time"][mask_filt].array, flux, err_flux, pspl_no_blend)
m = Minuit(c, *guess )

m.migrad()